In [1]:
import os
os.environ['HF_ENDPOINT'] = 'https://hf-mirror.com'

import os
from datasets import load_dataset
from evaluate import load
from transformers import (
    BertForSequenceClassification,
    BertTokenizerFast,
    Trainer,
    TrainingArguments,
    DataCollatorWithPadding
)
import numpy as np

In [4]:
def main():
    # 1. 加载数据集（假设已准备好 JSONL 格式）
    dataset = load_dataset(
        "json",
        data_files={"train": "train.jsonl",
                    "validation": "holdout.jsonl"}
    )

    # 2. 加载 tokenizer & model
    model_name = "/root/autodl-tmp/Day4/bert-base-chinese/"
    tokenizer  = BertTokenizerFast.from_pretrained(model_name)
    model      = BertForSequenceClassification.from_pretrained(model_name, num_labels=5)

    # 3. 数据预处理：tokenize
    label2id = {"财经": 0, "体育": 1, "娱乐": 2, "教育": 3, "科技": 4}
    id2label = {v: k for k, v in label2id.items()}

    # 把字符串 label 映射为数字
    def encode_labels(example):
        example["label"] = label2id[example["label"]]
        return example
    dataset = dataset.map(encode_labels)

    def tokenize_fn(batch):
        # 批量 tokenization
        return tokenizer(
            batch["text"],
            truncation=True,
            max_length=256
        )
    tokenized_ds = dataset.map(tokenize_fn, batched=True, remove_columns=["text"])

    # 4. 准备 DataCollator（动态 padding）
    data_collator = DataCollatorWithPadding(tokenizer)

    # 5. Metric：accuracy + macro F1
    accuracy = load("accuracy")
    f1       = load("f1")

    def compute_metrics(eval_pred):
        logits, labels = eval_pred
        preds = np.argmax(logits, axis=-1)
        acc   = accuracy.compute(predictions=preds, references=labels)["accuracy"]
        f1m   = f1.compute(predictions=preds, references=labels, average="macro")["f1"]
        return {"accuracy": acc, "f1": f1m}

    # 6. TrainingArguments
    training_args = TrainingArguments(
        output_dir="./bert_chinese_baseline",
        per_device_train_batch_size=16,
        per_device_eval_batch_size=32,
        learning_rate=2e-5,
        num_train_epochs=3,
        weight_decay=0.01,
        logging_dir="./logs",
        logging_steps=50,
        save_strategy="epoch",     # 老版只支持 save_strategy / save_steps
        # evaluation_strategy="epoch",  # 旧版不支持，注释掉
        # load_best_model_at_end=True,   # 注释掉
        # metric_for_best_model="f1",    # 注释掉
    )

    # 7. Trainer
    trainer = Trainer(
        model           = model,
        args            = training_args,
        train_dataset   = tokenized_ds["train"],
        eval_dataset    = tokenized_ds["validation"],
        tokenizer       = tokenizer,
        data_collator   = data_collator,
        compute_metrics = compute_metrics,
    )

    # 8. 训练
    trainer.train()

    # 9. 评估并输出
    results = trainer.evaluate()
    print("=== Validation Metrics ===")
    print(f"Accuracy: {results['eval_accuracy']:.4f}")
    print(f"Macro F1 : {results['eval_f1']:.4f}")

if __name__ == "__main__":
    main()

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at /root/autodl-tmp/Day4/bert-base-chinese/ and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Map:   0%|          | 0/9000 [00:00<?, ? examples/s]

Map:   0%|          | 0/1000 [00:00<?, ? examples/s]

Map:   0%|          | 0/9000 [00:00<?, ? examples/s]

Map:   0%|          | 0/1000 [00:00<?, ? examples/s]

/tmp/ipykernel_2456/1367164169.py:64: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(
/root/miniconda3/lib/python3.12/site-packages/torch/nn/modules/module.py:1747: FutureWarning: `encoder_attention_mask` is deprecated and will be removed in version 4.55.0 for `BertSdpaSelfAttention.forward`.
  return forward_call(*args, **kwargs)


Step,Training Loss
50,0.649000
100,0.140000
150,0.096800
200,0.107800
250,0.109300
300,0.071700
350,0.128600
400,0.100900
450,0.115700
500,0.066700


/root/miniconda3/lib/python3.12/site-packages/torch/nn/modules/module.py:1747: FutureWarning: `encoder_attention_mask` is deprecated and will be removed in version 4.55.0 for `BertSdpaSelfAttention.forward`.
  return forward_call(*args, **kwargs)
/root/miniconda3/lib/python3.12/site-packages/torch/nn/modules/module.py:1747: FutureWarning: `encoder_attention_mask` is deprecated and will be removed in version 4.55.0 for `BertSdpaSelfAttention.forward`.
  return forward_call(*args, **kwargs)
/root/miniconda3/lib/python3.12/site-packages/torch/nn/modules/module.py:1747: FutureWarning: `encoder_attention_mask` is deprecated and will be removed in version 4.55.0 for `BertSdpaSelfAttention.forward`.
  return forward_call(*args, **kwargs)


=== Validation Metrics ===
Accuracy: 0.9850
Macro F1 : 0.9847
